# RSNA MICCAI Brain Tumor Radiogenomic Classification

[<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/29653/logos/header.png?t=2021-07-07-17-26-56">](http://google.com.au/)

In this notebook I will try to classify the images using differente EfficientNet models. To deal with 3D data I will try several method:
* Aggregating data along the first axis
* Start with a 1x1 convolution to reduce dimensionality
* The 3D version of EfficientNet
* One model proposed [here](http://www.ajnr.org/content/42/5/845)

## Importing necessary libraries

In [ ]:
# Basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Neural network libraries
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.callbacks import EarlyStopping

# Reading images and creating video libraries
import cv2
from IPython.display import HTML
from base64 import b64encode
import matplotlib.animation as animation
import os

import SimpleITK as sitk

## Utility functions to visualize the images

I display a video with a collection of the images of each folder.

In [ ]:
def play(filename):
    html = ''
    video = open(filename,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=500 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

def create_video(imgs, output='/kaggle/working/predicted.mp4', duration=30, subplot=True, 
                frame_delay=200):
    fig, ax = plt.subplots(figsize=(15, 10))
    ims = []
    if not subplot:
        shape = imgs.shape[0]
        for i in range(duration):
            im = ax.imshow(imgs[i % shape], animated=True)
            ims.append([im])
        plt.close(fig)
    else:
        shapes = [imgs[views[0]].shape[0], imgs[views[1]].shape[0], 
                  imgs[views[2]].shape[0], imgs[views[3]].shape[0]]
        fig, ax = plt.subplots(2,2, figsize=(10,10))
        for k in range(duration):
            im_ = []
            for i in range(2):
                for j in range(2):
                    im = ax[i,j].imshow(imgs[views[2*i+j]][k % shapes[2*i+j]], animated=True)
                    im_.append(im)
                    ax[i,j].set_title(views[2*i+j])
                    plt.close()
            ims.append(im_)

    ani = animation.ArtistAnimation(fig, ims, interval=frame_delay, blit=True, repeat_delay=1000)

    ani.save(output)

## Labels

In [ ]:
target = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
preds = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')

## Read images utility function

In [ ]:
# specify your image path
views = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
def load_imgs(idx, ignore_zeros=True, train=True):
    imgs = {}
    for view in views:
        save_ds = []
        if train:
            dir_path = os.walk(os.path.join(
            '../input/rsna-miccai-png/train/', idx, view
        ))
        else:
            dir_path = os.walk(os.path.join(
            '../input/rsna-miccai-png/test/', idx, view
        ))
        for path, subdirs, files in dir_path:
            for name in files:
                image_path = os.path.join(path, name) 
                ds = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                save_ds.append(np.array(ds))
        if len(save_ds) == 0:
            save_ds = np.zeros((1,256,256))
        imgs[view] = np.array(save_ds)
    return imgs

Here we try loading 32 images to see how much it takes. This will be the base to set the batch size later on so that each iteration is less expensive in time.

In [ ]:
%%time
for i in range(32):
    idx = str(target.BraTS21ID[i]).zfill(5)
    imgs = load_imgs(idx)

Also, there are some folders without images. For those we simply define a zero-valued image so that the models work fine.

In [ ]:
# Pathological one
idx = str(109).zfill(5)
imgs = load_imgs(idx)

## Example of Image Visualization

In [ ]:
fig, ax = plt.subplots(2,2, figsize=(10,10))
for i in range(2):
    for j in range(2):
        m = ax[i,j].imshow(imgs[views[2*i+j]].mean(axis=0))
        ax[i,j].set_title(views[2*i+j])
plt.show()

# Example of Video Visualization

In [ ]:
create_video(imgs, duration=60, subplot=True, frame_delay=300)
play('predicted.mp4')

## DataGenerator

Since the data is massive, and it is a good practice to use them, I have created the data loaders for the models. The only thing to highlight is that we use N4BiasFieldCorrectionImageFilter to correct the bias of the images as shown in the paper I mentioned at the beginning.

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels=None, batch_size=256, dim=(512,512), n_channels=4,
                 n_classes=2, shuffle=True, is_train=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.is_train = (labels is not None)
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        list_IDs_temp = self.list_IDs[index*self.batch_size:(index+1)*self.batch_size]

        X = self.__data_generation(list_IDs_temp)
        # Generate data
        if self.is_train:
            y = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            return np.array(X), np.array(y)
        else:
            return np.array(X)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            idx = str(ID).zfill(5)
            imgs = load_imgs(idx, ignore_zeros=False, train=self.is_train)
            new_imgs = []
            for ii in range(2):
                for jj in range(2):
                    img_ = imgs[views[2*ii+jj]].mean(axis=0)
                    img_ = cv2.resize(img_, dsize=self.dim, interpolation=cv2.INTER_LINEAR)
                    img_ = np.array(img_, dtype='float32') 
                    
                    # Removing radiofrequency inhomogeneity using N4 Bias Field Correction 
                    inputImage = sitk.GetImageFromArray(img_)
                    maskImage = sitk.GetImageFromArray((img_ > 0.1) * 1)
                    inputImage = sitk.Cast(inputImage, sitk.sitkFloat32)
                    maskImage = sitk.Cast(maskImage, sitk.sitkUInt8)
                    corrector = sitk.N4BiasFieldCorrectionImageFilter()
                    numberFittingLevels = 4
                    maxIter = 100
                    if maxIter is not None:
                        corrector.SetMaximumNumberOfIterations([maxIter]
                                                               * numberFittingLevels)
                    corrected_image = corrector.Execute(inputImage, maskImage)
                    img_ = sitk.GetArrayFromImage(corrected_image)
                    new_imgs.append(img_)
            new_imgs = np.array(new_imgs).transpose(1,2,0)
            X[i,] = new_imgs
        
        return X

### Usual train-validation split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(target.BraTS21ID, target.MGMT_value,
                                                 test_size=0.2, random_state=0,
                                                 stratify=target.MGMT_value)

In [ ]:
dim = (256,256)
train_dataset = DataGenerator(X_train, y_train, batch_size=8, dim=dim)
val_dataset = DataGenerator(X_val, y_val, batch_size=8, dim=dim)
test_dataset = DataGenerator(preds.BraTS21ID, batch_size=8, dim=dim)

### EfficientNet

In [ ]:
!pip install efficientnet

### Model #1

We use a convolutional layer to change the number of channels to 3 as it is needed by the EfficientNet model. We use early stopping but each iteration takes so long that until now I haven't managed to make it converge. Apart from that, the model is saved at each epoch. For this approach, no more than 0.55 in AUC was achieved.

In [ ]:
import efficientnet.keras as efn

with tf.device('/gpu:0'):
    model = keras.Sequential([
        layers.Conv2D(3, kernel_size=1, input_shape=(*dim, 4), padding='same'),
        efn.EfficientNetB0(include_top=False, pooling='avg'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    earlyStopping = EarlyStopping(patience=2, min_delta=0.001, verbose=1)
    
    checkpoint_path = "training_/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    #model.load_weights(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC()]
        )
    
    #history = model.fit(train_dataset, validation_data=val_dataset,
                                 #epochs=10, callbacks=[earlyStopping, cp_callback])

In [ ]:
from keras.applications.resnet import ResNet50

with tf.device('/gpu:0'):
    model = keras.Sequential([
        layers.Conv2D(3, kernel_size=1, input_shape=(*dim, 4), padding='same'),
        ResNet50(include_top=False, pooling='avg'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    earlyStopping = EarlyStopping(patience=2, min_delta=0.001, verbose=1)
    
    checkpoint_path = "training_resnet/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    #model.load_weights(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC()]
        )
    
    #history = model.fit(train_dataset, validation_data=val_dataset,
                                 #epochs=10, callbacks=[earlyStopping, cp_callback])

## DataGenerator3D

In this data generator we don't apply the mean to reduce dimensionality and we normalize the data to be zero-mean and unit-variance.

In [ ]:
from sklearn.preprocessing import StandardScaler

class DataGenerator3D(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels=None, batch_size=256, dim=(512,512,512), n_channels=4,
                 n_classes=2, shuffle=True, is_train=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.is_train = (labels is not None)
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        list_IDs_temp = self.list_IDs[index*self.batch_size:(index+1)*self.batch_size]

        X = self.__data_generation(list_IDs_temp)
        # Generate data
        if self.is_train:
            y = self.labels[index*self.batch_size:(index+1)*self.batch_size]
            return np.array(X), np.array(y)
        else:
            return np.array(X)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            idx = str(ID).zfill(5)
            imgs = load_imgs(idx, ignore_zeros=False, train=self.is_train)
            new_imgs = []
            for ii in range(2):
                for jj in range(2):
                    img_ = imgs[views[2*ii+jj]]
                    img_ = np.array([cv2.resize(img_[i], dsize=(self.dim[1],self.dim[0]), interpolation=cv2.INTER_LINEAR) for i in range(img_.shape[0])])
                    img_ = np.array([cv2.resize(img_.transpose(1,2,0)[i], dsize=(self.dim[2],self.dim[1]), interpolation=cv2.INTER_LINEAR) for i in range(self.dim[0])])
                    
                    # Removing radiofrequency inhomogeneity using N4 Bias Field Correction 
                    for p in range(len(img_)):
                        inputImage = sitk.GetImageFromArray(img_[p])
                        maskImage = sitk.GetImageFromArray((img_[p] >0.1) * 1)
                        inputImage = sitk.Cast(inputImage, sitk.sitkFloat32)
                        maskImage = sitk.Cast(maskImage, sitk.sitkUInt8)
                        corrector = sitk.N4BiasFieldCorrectionImageFilter()
                        numberFittingLevels = 4
                        maxIter = 100
                        if maxIter is not None:
                            corrector.SetMaximumNumberOfIterations([maxIter]
                                                                   * numberFittingLevels)
                        corrected_image = corrector.Execute(inputImage, maskImage)
                        img_[p] = sitk.GetArrayFromImage(corrected_image)
                        
                    # Normalization
                    sc = StandardScaler()
                    img_ = np.array([sc.fit_transform(img_[i]) for i in range(img_.shape[0])])

                    new_imgs.append(img_)
            new_imgs = np.concatenate(new_imgs).transpose(1,2,0).reshape((*self.dim,-1))
            X[i,] = new_imgs
        
        return X

### Again train-validation split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(target.BraTS21ID, target.MGMT_value,
                                                 test_size=0.2, random_state=0,
                                                 stratify=target.MGMT_value)

In [ ]:
train_dataset = DataGenerator3D(X_train, y_train, batch_size=4, dim=(64,64,16))
val_dataset = DataGenerator3D(X_val, y_val, batch_size=4, dim=(64,64,16))
test_dataset = DataGenerator3D(preds.BraTS21ID, batch_size=4, dim=(64,64,16))

### Model #2

Instead of aggregating by one axis, apply a 2D convolutional layer to infer that aggregation and use EfficientNet as before.

In [ ]:
with tf.device('/gpu:0'):
    model = keras.Sequential([
        layers.InputLayer(input_shape=(128,128,128,4)),
        layers.Reshape((128,128,-1)),
        layers.Conv2D(3, kernel_size=1, padding='same'),
        efn.EfficientNetB0(include_top=False, pooling='avg'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    earlyStopping = EarlyStopping(patience=2, min_delta=0.001, verbose=1)
    
    checkpoint_path = "training_0/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    #model.load_weights(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC()]
        )
    
    #history = model.fit(train_dataset, validation_data=val_dataset,
                                 #epochs=10, callbacks=[earlyStopping, cp_callback])

In [ ]:
!pip install efficientnet_3D

### Model #3

Use the 3D version of EfficientNet, no other operation used.

In [ ]:
import efficientnet_3D.tfkeras as efn3d

with tf.device('/gpu:0'):
    model = keras.Sequential([
        layers.InputLayer(input_shape=(64,64,16,4)),
        layers.Conv3D(3, kernel_size=1, padding='same'),
        efn3d.EfficientNetB0(include_top=False, input_shape=(64,64,16,3), pooling='avg'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    
    earlyStopping = EarlyStopping(patience=2, min_delta=0.001, verbose=1)
    
    checkpoint_path = "training_1/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC()]
        )
    
    #history = model.fit(train_dataset, validation_data=val_dataset,
                                 #epochs=10, callbacks=[earlyStopping, cp_callback])

### Model #4

This is a version similar to that presented in the paper.

In [ ]:
with tf.device('/gpu:0'):
    def blockTD(inp):
        x = layers.BatchNormalization()(inp)
        x = layers.Activation('relu')(x)
        x = layers.Conv3D(16, kernel_size=5, padding='same')(x)
        x = layers.Dropout(0.2)(x)
        out = layers.MaxPooling3D(2)(x)
        return out
    
    def blockTU(inp):
        x = layers.BatchNormalization()(inp)
        x = layers.Activation('relu')(x)
        x = layers.Conv3DTranspose(16, kernel_size=5, strides=2, padding='same')(x)
        out = layers.Dropout(0.2)(x)
        return out
    
    def blockDense_(inp):
        x = layers.BatchNormalization()(inp)
        x = layers.Activation('relu')(x)
        x = layers.Conv3D(16, kernel_size=5, padding='same')(x)
        out = layers.Dropout(0.2)(x)
        return out
    
    def blockDense(inp):
        y = blockDense_(inp)
        x = layers.Concatenate()([inp, y])
        out = y
        for _ in range(3):
            y = blockDense_(x)
            out = layers.Concatenate()([out, y])
            x = layers.Concatenate()([x, y])
        out = layers.Concatenate()([out, x])
        y = blockDense_(x)
        out = layers.Concatenate()([out, y])
        return out
        
    def build_model():
        inp = keras.Input(shape=(64,64,16,4))
        y = blockDense(inp)
        x = layers.Concatenate()([inp, y])
        x1 = tf.identity(x)
        
        x = blockTD(x)
        y = blockDense(x)
        x = layers.Concatenate()([x, y])
        x0 = tf.identity(x)
        
        x = blockTD(x)
        y = blockDense(x)
        x = layers.Concatenate()([x, y])
        
        y = blockTD(x)
        y = blockDense(y)
        y = blockTU(y)
        x = layers.Concatenate()([x, y])
        
        y = blockTU(x)
        y = blockDense(y)
        x = layers.Concatenate()([x0, y])
        
        y = blockTU(x)
        y = blockDense(y)
        x = layers.Concatenate()([x1, y])
        
        y = blockDense(x)
        y = layers.GlobalMaxPooling3D()(y)
        y = layers.Dense(32, activation='relu')(y)
        out = layers.Dense(1, activation='sigmoid')(y)
        return keras.Model(inputs = inp, outputs = out)
    
    model = build_model()
    
    earlyStopping = EarlyStopping(patience=2, min_delta=0.001, verbose=1)
    
    checkpoint_path = "training_2/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    #model.load_weights(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
    
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-4), 
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC()]
        )
    
    #history = model.fit(train_dataset, validation_data=val_dataset,
                                 #epochs=10, callbacks=[earlyStopping, cp_callback])

In [ ]:
with tf.device('/gpu:0'):
    model = build_model()
    
    checkpoint_path = "training_2/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    #model.load_weights(checkpoint_path)
    
    model.compile(
        optimizer='adam', 
        loss='binary_crossentropy',
        metrics=[keras.metrics.AUC()]
        )
    
    #preds = model.predict(test_dataset)